# Signed Differences Ordinary Least Squares

In this Jupyter notebook, we demonstrate our solution in four different setups:
- Positive:
    - No Noise
    - With Noise
- Signed:
    - No Noise
    - With Noise

The data here is a difference matrix $D$ based on a vector $X$ of numbers which are either positive, or a mix of positive and negative numbers (hence the term "signed"). The difference matrix $D$ may or may not be contaminated with a factor of $10^{-5}$ of a vector of Gaussian noise. 

In each setup, we solve the problem using our proposed `SignedDifferencesOLS` solver using each of the following libraries: `CVXPY`, `scipy`, and `sklearn`.

We also measure the time needed for the solver to complete the problem. We take $n = 10$ in all experiments.

Before moving to the experiments, we also quickly demonstrate our vectorization implementations.

## Vectorization Implementations

In what follows, let $P$ be a square matrix of size $n = 5$.

In [25]:
import numpy as np
from numpy.random import randn

n = 5

P = np.round(randn(n, n), 2)

P

array([[-0.57,  0.01, -1.71,  1.47,  0.32],
       [ 1.58, -0.29, -0.33,  1.83, -0.03],
       [-1.68,  0.67,  1.36,  1.86,  0.61],
       [-0.86, -0.14,  0.87,  0.69,  0.85],
       [-0.14, -0.94, -0.21,  0.03, -0.  ]])

In [26]:
from utils.lin_alg import half_vectorize

### Upper Half Vectorization

#### Including The Diagonal

In [27]:
half_vectorize(P, triangular='upper', diag=True)


array([-0.57,  0.01, -1.71,  1.47,  0.32, -0.29, -0.33,  1.83, -0.03,
        1.36,  1.86,  0.61,  0.69,  0.85, -0.  ])

#### Excluding The Diagonal

In [28]:
half_vectorize(P, triangular='upper', diag=False)

array([ 0.01, -1.71,  1.47,  0.32, -0.33,  1.83, -0.03,  1.86,  0.61,
        0.85])

### Lower Half Vectorization

#### Including The Diagonal

In [29]:
half_vectorize(P, triangular='lower', diag=True)


array([-0.57,  1.58, -0.29, -1.68,  0.67,  1.36, -0.86, -0.14,  0.87,
        0.69, -0.14, -0.94, -0.21,  0.03, -0.  ])

#### Excluding The Diagonal

In [30]:
half_vectorize(P, triangular='lower', diag=False)

array([ 1.58, -1.68,  0.67, -0.86, -0.14,  0.87, -0.14, -0.94, -0.21,
        0.03])

## Signed Differences OLS Experiments

In what follows, we set $n = 10$ and define the three different solvers to be compared.

In [7]:
from solver import SignedDifferencesOLS
from utils.data_gen import generate_data

n = 10

diff_solver = {
    'cvxpy' : SignedDifferencesOLS('cvxpy'),
    'sklearn' : SignedDifferencesOLS('sklearn'),
    'scipy' : SignedDifferencesOLS('scipy')
}

### Data Generation

In [8]:
data = {
    'positive' : {
        'no_noise' : generate_data(n, positive=True),
        'noise' : generate_data(n, positive=True, noise=True)
    },
    'signed' : {
        'no_noise' : generate_data(n, positive=False),
        'noise' : generate_data(n, positive=False, noise=True)
    }
}

In [9]:
nums = {
    'positive' : {
        'no_noise' : data['positive']['no_noise'].nums,
        'noise' : data['positive']['noise'].nums
    },
    'signed' : {
        'no_noise' : data['signed']['no_noise'].nums,
        'noise' : data['signed']['noise'].nums
    }
}
diff_matrices = {
    'positive' : {
        'no_noise' : data['positive']['no_noise'].diff_matrix,
        'noise' : data['positive']['noise'].diff_matrix
    },
    'signed' : {
        'no_noise' : data['signed']['no_noise'].diff_matrix,
        'noise' : data['signed']['noise'].diff_matrix
    }
}

### Comparison Of The Solvers

In [10]:
from utils.printing import results_message, vectors_comparison

#### Positive, No Noise

In [11]:
nums_type = 'positive'
noise_presence = 'no_noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [12]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is:
    [0.62358598 1.06842574 0.28291727 0.42188553 0.43504838 0.23853265
 0.4454705  0.53182311 0.82884285 0.25973484]
    The norm of the residual is: 1.1436209663188637e-21.
    

    The sklearn-based solution is:
    [0.38505333 0.82989309 0.04438462 0.18335287 0.19651573 0.
 0.20693785 0.29329045 0.59031019 0.02120218]
    The norm of the residual is: 1.2216001053570885e-15.
    

    The scipy-based solution is:
    [0.38505333 0.82989309 0.04438462 0.18335287 0.19651573 0.
 0.20693785 0.29329045 0.59031019 0.02120218]
    The norm of the residual is: 2.4973638151140144e-16.
    


In [13]:
for lib in diff_solver.keys():
    print(
        vectors_comparison(
        X_hat[lib], X, lib
    )
    )


    The cvxpy-based OLS solution (X_hat) is:
    [0.62358598 1.06842574 0.28291727 0.42188553 0.43504838 0.23853265
 0.4454705  0.53182311 0.82884285 0.25973484]
    The original vector (X) was:
    [0.42966055 0.87450031 0.08899184 0.22796009 0.24112295 0.04460722
 0.25154507 0.33789767 0.63491741 0.0658094 ]
    The max-norm of their difference is: 0.1939254360684194.
    

    The sklearn-based OLS solution (X_hat) is:
    [0.38505333 0.82989309 0.04438462 0.18335287 0.19651573 0.
 0.20693785 0.29329045 0.59031019 0.02120218]
    The original vector (X) was:
    [0.42966055 0.87450031 0.08899184 0.22796009 0.24112295 0.04460722
 0.25154507 0.33789767 0.63491741 0.0658094 ]
    The max-norm of their difference is: 0.04460721948162416.
    

    The scipy-based OLS solution (X_hat) is:
    [0.38505333 0.82989309 0.04438462 0.18335287 0.19651573 0.
 0.20693785 0.29329045 0.59031019 0.02120218]
    The original vector (X) was:
    [0.42966055 0.87450031 0.08899184 0.22796009 0.24112295

##### Observations
- Note that the `cvxpy`-based solution performs better than the other two.
- In some cases, the `scipy`-based and the `sklearn`-based solutions are identical. (Similar/identical implementations under the hood, perhaps?)
- Although the `cvxpy`-based solution produces the solution with the smallest residual norm (by orders of magnitude), the other two solutions are much closer to the original vector $X$ by order of magnitude.

In [40]:
nums_type = 'positive'
noise_presence = 'noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [41]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is: [0.74070371 0.39567285 0.81576167 1.04869095 0.71177126 0.80741857
 0.83174724 0.36637844 1.2106523  1.21636703].
    The norm of the residual is: 1.5929376680534574e-07.
    

    The sklearn-based solution is: [0.37416397 0.02915616 0.44926801 0.68222034 0.34532369 0.44099404
 0.46534575 0.         0.8442969  0.85003467].
    The norm of the residual is: 0.0007728852602550966.
    

    The scipy-based solution is: [0.37432527 0.02929441 0.44938323 0.68231251 0.34539282 0.44104013
 0.4653688  0.         0.84427386 0.84998859].
    The norm of the residual is: 0.0003991162321999125.
    


In [42]:
nums_type = 'signed'
noise_presence = 'no_noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [43]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is: [1.55385008 2.48665424 3.95925006 2.62433159 3.12565861 1.09476275
 4.20043732 1.44944501 3.20826022 3.36200069].
    The norm of the residual is: 1.9876961056974538e-20.
    

    The sklearn-based solution is: [0.45908731 1.39189142 2.86448716 1.52956876 2.03089576 0.
 3.10567442 0.35468224 2.11349736 2.26723783].
    The norm of the residual is: 7.992882943171108e-15.
    

    The scipy-based solution is: [0.45908731 1.39189142 2.86448716 1.52956876 2.03089576 0.
 3.10567442 0.35468224 2.11349736 2.26723783].
    The norm of the residual is: 2.4490890699602997e-15.
    


In [44]:
nums_type = 'positive'
noise_presence = 'no_noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [ ]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is: [0.69694447 0.58659862 0.89435223 1.01030242 0.5710543  0.68654638
 0.26169359 0.63305442 0.49160716 0.92719282].
    The norm of the residual is: 8.734208231075047e-22.
    

    The sklearn-based solution is: [0.43525089 0.32490503 0.63265864 0.74860884 0.30936072 0.4248528
 0.         0.37136083 0.22991357 0.66549923].
    The norm of the residual is: 1.969668317248782e-15.
    

    The scipy-based solution is: [0.43525089 0.32490503 0.63265864 0.74860884 0.30936072 0.4248528
 0.         0.37136083 0.22991357 0.66549923].
    The norm of the residual is: 9.316605982632408e-16.
    


#### Positive, Noise

In [14]:
nums_type = 'positive'
noise_presence = 'noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [15]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is:
    [0.81183992 0.967386   1.038925   0.41001198 0.45848949 0.73031176
 0.25820401 0.39572738 0.75194365 0.54088415]
    The norm of the residual is: 1.1506264303879066e-07.
    

    The sklearn-based solution is:
    [0.5535184  0.70908406 0.78064264 0.15174921 0.20024631 0.47208816
 0.         0.13754295 0.4937788  0.28273889]
    The norm of the residual is: 0.0006568751109571939.
    

    The scipy-based solution is:
    [0.5536359  0.70918198 0.78072098 0.15180797 0.20028548 0.47210775
 0.         0.13752337 0.49373963 0.28268014]
    The norm of the residual is: 0.0003392088487034717.
    


In [16]:
for lib in diff_solver.keys():
    print(
        vectors_comparison(
        X_hat[lib], X, lib
    )
    )


    The cvxpy-based OLS solution (X_hat) is:
    [0.81183992 0.967386   1.038925   0.41001198 0.45848949 0.73031176
 0.25820401 0.39572738 0.75194365 0.54088415]
    The original vector (X) was:
    [0.71443864 0.8700043  0.94156289 0.31266946 0.36116655 0.63300841
 0.16092024 0.2984632  0.65469905 0.44365914]
    The max-norm of their difference is: 0.09740127635145757.
    

    The sklearn-based OLS solution (X_hat) is:
    [0.5535184  0.70908406 0.78064264 0.15174921 0.20024631 0.47208816
 0.         0.13754295 0.4937788  0.28273889]
    The original vector (X) was:
    [0.71443864 0.8700043  0.94156289 0.31266946 0.36116655 0.63300841
 0.16092024 0.2984632  0.65469905 0.44365914]
    The max-norm of their difference is: 0.16092024439308505.
    

    The scipy-based OLS solution (X_hat) is:
    [0.5536359  0.70918198 0.78072098 0.15180797 0.20028548 0.47210775
 0.         0.13752337 0.49373963 0.28268014]
    The original vector (X) was:
    [0.71443864 0.8700043  0.94156289 0.31

##### Observations
- Once again, the `cvxpy`-based solution performs better than the other two.
    - This time, however, the difference in residual norm is not staggering.
- Once again, the `scipy`-based and the `sklearn`-based solutions are identical.
- Unlike the previous experiment, the `cvxpy`-based solution is closer to the original vector $X$ than the other two solutions.

#### Signed, No Noise

In [17]:
nums_type = 'signed'
noise_presence = 'no_noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [18]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is:
    [1.32836184 1.74156221 1.71569584 3.31394428 3.21919259 0.86580543
 2.53133878 2.02334307 3.25204675 1.81515595]
    The norm of the residual is: 1.281683829121875e-20.
    

    The sklearn-based solution is:
    [0.46255639 0.87575673 0.84989037 2.44813872 2.35338704 0.
 1.66553327 1.15753758 2.3862412  0.94935047]
    The norm of the residual is: 8.430798925816896e-15.
    

    The scipy-based solution is:
    [0.46255639 0.87575673 0.84989037 2.44813872 2.35338704 0.
 1.66553327 1.15753758 2.3862412  0.94935047]
    The norm of the residual is: 2.3644026431296272e-15.
    


In [19]:
for lib in diff_solver.keys():
    print(
        vectors_comparison(
        X_hat[lib], X, lib
    )
    )


    The cvxpy-based OLS solution (X_hat) is:
    [1.32836184 1.74156221 1.71569584 3.31394428 3.21919259 0.86580543
 2.53133878 2.02334307 3.25204675 1.81515595]
    The original vector (X) was:
    [-0.7244907  -0.31129036 -0.33715672  1.26109163  1.16633995 -1.18704709
  0.47848618 -0.02950951  1.19919411 -0.23769662]
    The max-norm of their difference is: 2.052852641680211.
    

    The sklearn-based OLS solution (X_hat) is:
    [0.46255639 0.87575673 0.84989037 2.44813872 2.35338704 0.
 1.66553327 1.15753758 2.3862412  0.94935047]
    The original vector (X) was:
    [-0.7244907  -0.31129036 -0.33715672  1.26109163  1.16633995 -1.18704709
  0.47848618 -0.02950951  1.19919411 -0.23769662]
    The max-norm of their difference is: 1.187047089145204.
    

    The scipy-based OLS solution (X_hat) is:
    [0.46255639 0.87575673 0.84989037 2.44813872 2.35338704 0.
 1.66553327 1.15753758 2.3862412  0.94935047]
    The original vector (X) was:
    [-0.7244907  -0.31129036 -0.33715672  

##### Observations
- Once again, the `cvxpy`-based solution performs better than the other two (by orders of magnitude).
- Once again, the `scipy`-based and the `sklearn`-based solutions are identical.
- As before, although the `cvxpy`-based solution produces the solution with the smallest residual norm, the other two solutions are much closer to the original vector $X$. With that being said, the difference is not as large as before.

#### Signed, Noise

In [21]:
nums_type = 'signed'
noise_presence = 'noise'

X = nums[nums_type][noise_presence]
D = diff_matrices[nums_type][noise_presence]

X_hat = {
    lib : diff_solver[lib].solve(D).sol for lib in diff_solver.keys()
}
res_norm = {
    lib : diff_solver[lib].solve(D).res_norm for lib in diff_solver.keys()
}

In [22]:
for lib in diff_solver.keys():
    print(
        results_message(
        X_hat[lib], res_norm[lib], lib
    )
    )


    The cvxpy-based solution is:
    [1.1458228  2.37753618 4.90626082 3.00741346 3.52743122 4.06116264
 1.41958303 2.6386527  2.01213112 2.70718848]
    The norm of the residual is: 8.368059772312033e-08.
    

    The sklearn-based solution is:
    [0.         1.23173002 3.76047124 1.86164067 2.38167511 2.91542321
 0.27386042 1.49294674 0.86644189 1.56151592]
    The norm of the residual is: 0.0005601805436237971.
    

    The scipy-based solution is:
    [0.         1.23171332 3.76043784 1.86159057 2.3816083  2.9153397
 0.27376021 1.49282983 0.86630828 1.5613656 ]
    The norm of the residual is: 0.0002892759888465662.
    


In [23]:
for lib in diff_solver.keys():
    print(
        vectors_comparison(
        X_hat[lib], X, lib
    )
    )


    The cvxpy-based OLS solution (X_hat) is:
    [1.1458228  2.37753618 4.90626082 3.00741346 3.52743122 4.06116264
 1.41958303 2.6386527  2.01213112 2.70718848]
    The original vector (X) was:
    [-1.85869937 -0.62696935  1.90177187  0.0029413   0.52297574  1.05672384
 -1.58483895 -0.36575264 -0.99225748 -0.29718345]
    The max-norm of their difference is: 3.004522170621504.
    

    The sklearn-based OLS solution (X_hat) is:
    [0.         1.23173002 3.76047124 1.86164067 2.38167511 2.91542321
 0.27386042 1.49294674 0.86644189 1.56151592]
    The original vector (X) was:
    [-1.85869937 -0.62696935  1.90177187  0.0029413   0.52297574  1.05672384
 -1.58483895 -0.36575264 -0.99225748 -0.29718345]
    The max-norm of their difference is: 1.8586993712217084.
    

    The scipy-based OLS solution (X_hat) is:
    [0.         1.23171332 3.76043784 1.86159057 2.3816083  2.9153397
 0.27376021 1.49282983 0.86630828 1.5613656 ]
    The original vector (X) was:
    [-1.85869937 -0.626969

##### Observations
- Note that the `cvxpy`-based solution performs better than the other two.
- In some cases, the `scipy`-based and the `sklearn`-based solutions are identical. (Similar/identical implementations under the hood, perhaps?)
- Once again, although the `cvxpy`-based solution produces the solution with the smallest residual norm (by orders of magnitude), the other two solutions are much closer to the original vector $X$ by order of magnitude.
    - In particular, in this case, the `cvxpy` solution is still fairly noise-robust.